## Looking at Stuttered Pairs

Craig Lage - 02-Nov-22

In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from lsst.summit.utils import BestEffortIsr, makeDefaultLatissButler
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
from lsst.pipe.tasks.background import CharacterizeImageTask, CharacterizeImageConfig
%matplotlib inline

In [ ]:
butler = makeDefaultLatissButler(oga=True)
dataId = {'day_obs': 20221012, 'seq_num': 472, 'detector': 0, "instrument": 'LATISS'}
exp = butler.get('quickLookExp', dataId=dataId)

In [ ]:
charConfig = CharacterizeImageConfig()
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.doDeblend = False
charConfig.repair.doCosmicRay = True
charConfig.repair.doInterpolate = False
charConfig.detection.minPixels = 10
charTask = CharacterizeImageTask(config=charConfig)

charResult = charTask.run(exp)
sourceCatalog = charResult.sourceCat


In [ ]:
cat = sourceCatalog
plt.figure(figsize=(16,16))
arr = exp.image.array
arr = np.clip(arr, 1, 100000) # This image has some negative values, and this removes them
img = plt.imshow(arr, norm=LogNorm(vmin=1, vmax=100000),  interpolation='Nearest', cmap='gray')

plt.scatter(cat['base_SdssCentroid_x'],cat['base_SdssCentroid_y']\
            ,color='red', marker='x')

#plt.ylim(0,4000)
#plt.savefig(f'/project/cslage/AuxTel/offsets/{expId}.png')

In [ ]:
from lsst.summit.utils.utils import detectObjectsInExp

In [ ]:
objects = detectObjectsInExp(exp, nSigma=5)

In [ ]:
detectObjectsInExp?

In [ ]:
len(objects.getFootprints())

In [ ]:
cat = sourceCatalog
plt.figure(figsize=(16,16))
arr = exp.image.array
arr = np.clip(arr, 1, 100000) # This image has some negative values, and this removes them
img = plt.imshow(arr, norm=LogNorm(vmin=1, vmax=100000),  interpolation='Nearest', cmap='gray')
for i in range(len(objects.getFootprints())):
    plt.scatter(objects.getFootprints()[i].getCentroid().x,objects.getFootprints()[i].getCentroid().y\
            ,color='red', marker='x')

#plt.ylim(0,4000)
#plt.savefig(f'/project/cslage/AuxTel/offsets/{expId}.png')

In [ ]:
objects.getFootprints()[7].getCentroid().x

In [ ]:
plt.figure(figsize=(4,8))
plotCounter = 1
for x in range(1870, 1890, 1):
    plt.subplot(20,1,plotCounter)
    plt.plot(arr[0:2000, x])
    plt.ylim(0,200000)
    plotCounter += 1

In [ ]:
plt.plot(arr[0:2000, 1880])

In [ ]:
maxFlux = np.nanmax(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'])
selectBrightestSource = sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.99
brightestSource = sourceCatalog.subset(selectBrightestSource)
brightestCentroid = (brightestSource['base_SdssCentroid_x'][0], \
                     brightestSource['base_SdssCentroid_y'][0])
brightCatalog = sourceCatalog.subset(sourceCatalog['base_CircularApertureFlux_3_0_instFlux'] > maxFlux * 0.001)
print(f"expId:{expId}. Found {len(sourceCatalog)} sources, {len(brightCatalog)} bright sources")
print(f"Brightest centroid at {brightestCentroid}")
